In [13]:
!pip install sentence_transformers==2.7.0
!pip install pandas
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 5.9 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
len(model.encode("This is a test sentence"))

768

In [7]:
import pandas as pd
df = pd.read_csv("/content/travel_data.csv")

In [8]:
df.insert(0, 'id', range(1, len(df) + 1))
df.head()

,id,destination,user_review,travel_tip,rating,best_time_to_visit,local_cuisine_highlights,location_coordinates,popular_attractions,transportation_options,language_spoken,safety_rating,activities_available,cultural_highlights
0,1,"paris, france",Visiting the Eiffel Tower was breathtaking! Th...,Book tickets online to avoid long lines!,5,"April to June, September","Croissants, Macarons","48.8584° N, 2.2945° E","Louvre, Notre-Dame, Montmartre","Metro, Buses, Walking",French,4,"Seine River Cruises, Art Tours","Fashion, Art, Romantic Spots"
1,2,"tokyo, japan",Shibuya Crossing is a must-see. It's an incred...,Visit in the evening for the best lights!,4,"March to May, October","Sushi, Ramen","35.6586° N, 139.7012° E","Senso-ji, Tokyo Tower","Train, Metro, Buses",Japanese,5,"Temple Visits, Food Tours","Technology, Anime Culture"
2,3,"new_york, usa",Central Park is a great escape from the city h...,Rent a bike to explore more of the park.,5,Spring and Fall,"Bagels, Cheesecake","40.7851° N, 73.9683° W","Statue of Liberty, Times Square","Subway, Buses, Taxis",English,4,"Broadway Shows, City Tours","Diverse Cultures, Art Scene"
3,4,"rome, italy",The Colosseum is stunning! Make sure to take a...,Visit early in the morning to avoid crowds.,5,"April to June, September","Pasta Carbonara, Gelato","41.8902° N, 12.4923° E","Vatican City, Pantheon","Metro, Buses, Walking",Italian,4,"Historical Tours, Cooking Classes","Ancient History, Religion"
4,5,"cape_town, south_africa",Table Mountain offers the best views of the ci...,Take the cable car if you’re short on time.,4,November to February,"Bobotie, Biltong","-33.9625° S, 18.4095° E","Cape of Good Hope, Robben Island","Car, Taxi, Walking","Afrikaans, English",4,"Wine Tasting, Hiking","Cultural Heritage, Nature"


In [9]:
documents = df.to_dict(orient='records')
documents[0]

{'id': 1,
 'destination': 'paris, france',
 'user_review': 'Visiting the Eiffel Tower was breathtaking! The views from the top are unforgettable.',
 'travel_tip': 'Book tickets online to avoid long lines!',
 'rating': 5,
 'best_time_to_visit': 'April to June, September',
 'local_cuisine_highlights': 'Croissants, Macarons',
 'location_coordinates': '48.8584° N, 2.2945° E',
 'popular_attractions': 'Louvre, Notre-Dame, Montmartre',
 'transportation_options': 'Metro, Buses, Walking',
 'language_spoken': 'French',
 'safety_rating': 4,
 'activities_available': 'Seine River Cruises, Art Tours',
 'cultural_highlights': 'Fashion, Art, Romantic Spots'}

In [11]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["destination"]).tolist()
    operations.append(doc)

In [14]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://54.144.217.33:9200')

es_client.info()


ObjectApiResponse({'name': '020cfa9019a5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'U--Sn6YpSTmhWg7qQtl38w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
          "id": {"type": "integer"},
            "destination": {"type": "text"},
            "user_review": {"type": "text"},
            "travel_tip": {"type": "text"},
            "rating": {"type": "integer"},
            "best_time_to_visit": {"type": "text"},
            "local_cuisine_highlights": {"type": "text"},
            "location_coordinates": {"type": "text"},
            "popular_attractions": {"type": "text"},
            "transportation_options": {"type": "text"},
            "language_spoken": {"type": "text"},
            "safety_rating": {"type": "integer"},
            "activities_available": {"type": "text"},
            "cultural_highlights": {"type": "text"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}

        }
    }
}

In [23]:
index_name = "travel-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'travel-questions'})

In [24]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [25]:
search_term = "where to travel in sydney?"
vector_search_term = model.encode(search_term)

In [26]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
}


In [27]:
res = es_client.search(index=index_name, knn=query, source=[
                        "destination",
                        "user_review",
                        "travel_tip",
                        "best_time_to_visit",
                        "local_cuisine_highlights",
                        "location_coordinates",
                        "popular_attractions",
                        "transportation_options",
                        "language_spoken",
                        "activities_available",
                        "cultural_highlights"
                    ])
res["hits"]["hits"]

[{'_index': 'travel-questions',
  '_id': '1lrz0pIBDiocvnGqnqgQ',
  '_score': 0.76437986,
  '_source': {'local_cuisine_highlights': 'Meat Pies, Lamingtons',
   'cultural_highlights': 'Outdoor Lifestyle, Multicultural',
   'user_review': 'Bondi Beach is beautiful! The coastal walk from Bondi to Coogee is a highlight.',
   'destination': 'sydney, australia',
   'popular_attractions': 'Sydney Opera House, Harbour Bridge',
   'transportation_options': 'Train, Bus, Ferry',
   'language_spoken': 'English',
   'travel_tip': 'Bring sunscreen; the sun can be intense!',
   'location_coordinates': '-33.8908° S, 151.2743° E',
   'activities_available': 'Surfing, Wildlife Tours',
   'best_time_to_visit': 'December to February'}},
 {'_index': 'travel-questions',
  '_id': 'RFrz0pIBDiocvnGqu6nI',
  '_score': 0.73812497,
  '_source': {'local_cuisine_highlights': 'Meat Pie, Tim Tams',
   'cultural_highlights': 'Art Scene, Multiculturalism',
   'user_review': 'The coffee culture is vibrant!',
   'destinat